In [1]:
# install packages in libs (as editable)
!python -m pip install -e ../libs

Obtaining file:///Users/n0c09jf/code/github/binsense/libs
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for binsense-libs (pyproject.toml) ... done
  Created wheel for binsense-libs: filename=binsense_libs-0.0.1-py2.py3-none-any.whl size=1103 sha256=ac463b717126577342b3a2d53ca2f88cc47fb834e1862f1e8e0fa1cffa54420c
  Stored in directory: /private/var/folders/vm/2g2pvy116zzglvllmm0n_f100000gx/T/pip-ephem-wheel-cache-yo2j7qo5/wheels/77/72/16/1e76e1dcd0e567ae2a1096c6d731674e5dee84ed8ab15b770d
Successfully built binsense-libs


In [1]:
from binsense.config import DATA_DIR
print(DATA_DIR)

/Users/n0c09jf/code/github/binsense/_data
